Bitswan Gitops Webhook Pipeline
-------------------------------

This pipeline allows you to easilly deploy Bitswan pipelines to any server. You just need to POST to the webhook in your CI and your pipelines will be running in no time.

To install bitswan you need to initialize a git repository on your server/the place you want to run your bitswan installation and run the bitswan-pre docker image with the following ENV vars set:

- `BS_WEBHOOK_PORT`
- `BS_WEBHOOK_SECRET` - A secret that will be added to your webhook URL. To deploy post `{"action": "deploy-git"}` to the URL `https://localhost:<BS_WEBHOOK_PORT>/?secret=<BS_WEBHOOK_SECRET>`
- `BS_REPO_ROOT` - The repo with your data machines.
- `BS_BITSWAN_DIR` - The directory where your `bitswan.yaml` file resides. Should be a subdirectory of `BS_REPO_ROOT`

Simply adding the following curl command to your CI/CD pipeline should be enough to automatically deploy your data machines:

```
curl -X POST "https://<WEBHOOK_URL>:<BS_WEBHOOK_PORT>/?secret=<BS_WEBHOOK_SECRET>" -d '{"action": "deploy-git"}'
```

In [1]:
from bspump.jupyter import *
import bspump.http.web.source
import bspump.file
import time
import json
import os
import asyncio
import yaml

In [3]:
new_pipeline("BitswanGitopsWebHookPipeline")

In [4]:
@register_source
def source(app, pipeline):
   return bspump.http.web.source.WebHookSource(
       app,
       pipeline,
       config = {
           "port": os.environ.get("BS_WEBHOOK_PORT", 8080),
           "path": "/",
           "secret_qparam": os.environ.get("BS_WEBHOOK_SECRET", "not-secure")
       })

In [5]:
sample_events([
    b"""{"action": "deploy-git"}"""
])

In [6]:
@step
def parse_json(event):
    return json.loads(event)

{'action': 'deploy-git'}


In [ ]:
@step
def get_repo_root(event):
    repo_root = os.environ.get("BS_REPO_ROOT", "/mnt/repo")
    return repo_root

In [2]:
sample_events([
    "/home/timothy/pr/libertyaces/product/bitswanmonorepo/server-configs/libertyaces/hetzner-1/docker-compose/zabbix-demo"
])

In [ ]:
@async_step
async def git_pull(inject, repo_root):
    await asyncio.create_subprocess_exec("git", "pull", cwd=repo_root)
    await inject(repo_root)

In [4]:
@step
def load_bitswan_yaml(repo_root):
    dmy = yaml.full_load(open(os.path.join(repo_root, "bitswan.yaml")))
    return (repo_root, dmy)

('/home/timothy/pr/libertyaces/product/bitswanmonorepo/server-configs/libertyaces/hetzner-1/docker-compose/zabbix-demo', {'env-dir': '/etc/data-machines-secrets/', 'default-networks': ['default'], 'data-machines': {'demo-zabbix-demo-loader': {'volumes': ['/mnt/data/zabbix-demo/:/data/'], 'source': 'demo-zabbix-demo-loader'}, 'demo-zabbix-cleaner': {'source': 'zabbix-cleaner'}, 'demo-zabbix-analyzer': {'source': 'zabbix-analyzer'}, 'demo-zabbix-shoveler': {'source': 'zabbix-shoveler'}}})


In [9]:
@step
def generate_docker_compose(event):
    (repo_root, bitswan_yaml) = event
    dc = {
        "version": 3,
        "services": {}
    }
    for deployment_id, conf in bitswan_yaml["data-machines"].items():
        entry = {}
        entry["ENVIRONMENT"] = {"DEPLOYMENT_ID": deployment_id}
        if "env-dir" in bitswan_yaml:
           env_file = os.path.join(bitswan_yaml["env-dir"], deployment_id)
           if os.path.exists(env_file):
               entry["env_file"] = [env_file]
        if "default-networks" in bitswan_yaml:
            entry["networks"] = bitswan_yaml["default-networks"].copy()
        dc["services"][deployment_id] = entry
    dc_yaml = yaml.dump(dc)
    print(dc_yaml)
    return (repo_root, dc_yaml)

services:
  demo-zabbix-analyzer:
    ENVIRONMENT:
      DEPLOYMENT_ID: demo-zabbix-analyzer
    env_file:
    - /etc/data-machines-secrets/demo-zabbix-analyzer
    networks:
    - default
  demo-zabbix-cleaner:
    ENVIRONMENT:
      DEPLOYMENT_ID: demo-zabbix-cleaner
    networks:
    - default
  demo-zabbix-demo-loader:
    ENVIRONMENT:
      DEPLOYMENT_ID: demo-zabbix-demo-loader
    networks:
    - default
  demo-zabbix-shoveler:
    ENVIRONMENT:
      DEPLOYMENT_ID: demo-zabbix-shoveler
    networks:
    - default
version: 3

('/home/timothy/pr/libertyaces/product/bitswanmonorepo/server-configs/libertyaces/hetzner-1/docker-compose/zabbix-demo', 'services:\n  demo-zabbix-analyzer:\n    ENVIRONMENT:\n      DEPLOYMENT_ID: demo-zabbix-analyzer\n    env_file:\n    - /etc/data-machines-secrets/demo-zabbix-analyzer\n    networks:\n    - default\n  demo-zabbix-cleaner:\n    ENVIRONMENT:\n      DEPLOYMENT_ID: demo-zabbix-cleaner\n    networks:\n    - default\n  demo-zabbix-demo-loader:\n 

In [10]:
@step
def get_bitswan_dir(event):
    (repo_root, docker_compose_yaml) = event
    bitswan_dir = os.environ.get("BS_BITSWAN_DIR", "/mnt/repo/bitswan")
    return repo_root, bitswan_dir, docker_compose_yaml

('/home/timothy/pr/libertyaces/product/bitswanmonorepo/server-configs/libertyaces/hetzner-1/docker-compose/zabbix-demo', '/mnt/repo/bitswan', 'services:\n  demo-zabbix-analyzer:\n    ENVIRONMENT:\n      DEPLOYMENT_ID: demo-zabbix-analyzer\n    env_file:\n    - /etc/data-machines-secrets/demo-zabbix-analyzer\n    networks:\n    - default\n  demo-zabbix-cleaner:\n    ENVIRONMENT:\n      DEPLOYMENT_ID: demo-zabbix-cleaner\n    networks:\n    - default\n  demo-zabbix-demo-loader:\n    ENVIRONMENT:\n      DEPLOYMENT_ID: demo-zabbix-demo-loader\n    networks:\n    - default\n  demo-zabbix-shoveler:\n    ENVIRONMENT:\n      DEPLOYMENT_ID: demo-zabbix-shoveler\n    networks:\n    - default\nversion: 3\n')


In [ ]:
@async_step
async def docker_compose_up_daemon(inject, event):
    (repo_root, bitswan_dir, docker_compose_yaml) = event
    cmd = ["docker-compose", "-f", "-", "up", "-d"]

    # Create a subprocess with stdin pipe
    proc = await asyncio.create_subprocess_exec(
        *cmd, 
        stdin=asyncio.subprocess.PIPE, 
        stdout=asyncio.subprocess.PIPE, 
        stderr=asyncio.subprocess.PIPE,
        cwd=bitswan_dir
    )

    # Send docker_compose_yaml as input to the process and wait for completion
    stdout, stderr = await proc.communicate(input=docker_compose_yaml.encode())

    event = {
        "cmd": cmd,
        "stdout": stdout.decode("utf-8"),
        "stderr": stderr.decode("utf-8"),
        "returncode": proc.returncode,
    }
    await inject(event)

In [ ]:
@step
def serialize_json(event):
    return json.dumps(event)

In [8]:
@register_sink
def init_sink(app, pipeline):
    return bspump.file.FileBlockSink(app, pipeline)

In [9]:
end_pipeline()